# Conversion microbenchmarks

This notebook times conversions between QuASAr's tableau, decision diagram, and statevector
representations. It reproduces the `scripts/bench_conversion.py` sweep while adding fidelity-aware
statevector truncation experiments and figure export helpers.


## Environment and dependencies

The experiments rely on Qiskit, Qiskit Aer, Stim, and (optionally) MQT DDSIM. Install requirements
via `pip install -r requirements.txt`. Install `pandas` for convenient CSV handling if it is not
already available.


In [ ]:
from __future__ import annotations

import json
import math
import time
from pathlib import Path
import sys
from typing import Dict, Iterable, List, Optional, Sequence

import numpy as np
try:
    import pandas as pd
except ImportError:  # pragma: no cover - optional dependency
    pd = None
import matplotlib.pyplot as plt

REPO_ROOT = Path.cwd()
if (REPO_ROOT / 'scripts').exists() and str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford

from scripts.bench_conversion import _build_random_clifford_circuit
from quasar.conversion.tab2sv import tableau_to_statevector
from quasar.conversion.tab2dd import tableau_to_dd
from quasar.conversion.dd2sv import dd_to_statevector
from quasar.backends.dd import ddsim_available

plt.rcParams.update({'figure.figsize': (7.5, 4.5), 'axes.grid': True})


In [ ]:
def truncate_statevector(vector: np.ndarray, fidelity_target: float) -> Dict[str, float | np.ndarray]:
    """Return the truncated vector that meets (or exceeds) the fidelity target.

    The procedure keeps the largest amplitudes until the cumulative probability crosses the
    requested fidelity. The returned fidelity is computed after renormalisation.
    """

    fidelity_target = float(np.clip(fidelity_target, 0.0, 1.0))
    magnitudes = np.abs(vector) ** 2
    order = np.argsort(magnitudes)[::-1]
    cumulative = np.cumsum(magnitudes[order])
    threshold_index = int(np.searchsorted(cumulative, fidelity_target, side='left'))
    keep = order[: threshold_index + 1]

    truncated = np.zeros_like(vector)
    truncated[keep] = vector[keep]
    norm = np.linalg.norm(truncated)
    if norm == 0.0:
        return {
            'vector': truncated,
            'fidelity': 0.0,
            'nonzero': 0,
        }
    truncated /= norm
    fidelity = float(np.abs(np.vdot(vector, truncated)) ** 2)
    return {
        'vector': truncated,
        'fidelity': fidelity,
        'nonzero': int(np.count_nonzero(truncated)),
    }


def _benchmark_single_configuration(
    num_qubits: int,
    depth: int,
    repeat_index: int,
    fidelity_targets: Sequence[float],
    *,
    rng: np.random.Generator,
) -> List[Dict[str, object]]:
    circuit_seed = int(rng.integers(0, np.iinfo(np.int32).max))
    local_rng = np.random.default_rng(circuit_seed)
    circuit = _build_random_clifford_circuit(num_qubits, depth, local_rng)
    clifford = Clifford(circuit)

    rows: List[Dict[str, object]] = []
    metadata = {
        'num_qubits': num_qubits,
        'depth': depth,
        'repeat': repeat_index,
        'circuit_seed': circuit_seed,
        'circuit_depth': int(circuit.depth()),
        'two_qubit_ops': int(circuit.num_nonlocal_gates()),
    }

    start = time.perf_counter()
    try:
        tableau_state = tableau_to_statevector(clifford)
        success = True
        error = None
    except Exception as exc:  # pragma: no cover - runtime dependency guard
        tableau_state = None
        success = False
        error = repr(exc)
    elapsed = time.perf_counter() - start
    rows.append({**metadata, 'conversion': 'tableau_to_sv', 'elapsed_s': elapsed, 'success': success, 'error': error})

    start = time.perf_counter()
    try:
        dd_candidate = tableau_to_dd(clifford)
        dd_success = dd_candidate is not None
        dd_error = None
    except Exception as exc:
        dd_candidate = None
        dd_success = False
        dd_error = repr(exc)
    dd_elapsed = time.perf_counter() - start
    rows.append({**metadata, 'conversion': 'tableau_to_dd', 'elapsed_s': dd_elapsed, 'success': dd_success, 'error': dd_error})

    if dd_success:
        start = time.perf_counter()
        try:
            dd_state = dd_to_statevector(dd_candidate)
            dd_sv_success = dd_state is not None
            dd_sv_error = None
        except Exception as exc:
            dd_state = None
            dd_sv_success = False
            dd_sv_error = repr(exc)
        dd_sv_elapsed = time.perf_counter() - start
        rows.append({
            **metadata,
            'conversion': 'dd_to_sv',
            'elapsed_s': dd_sv_elapsed,
            'success': dd_sv_success,
            'error': dd_sv_error,
            'ddsim_available': ddsim_available(),
        })
    else:
        rows.append({
            **metadata,
            'conversion': 'dd_to_sv',
            'elapsed_s': None,
            'success': False,
            'error': 'tableau_to_dd failed',
            'ddsim_available': ddsim_available(),
        })

    if isinstance(tableau_state, np.ndarray):
        for fidelity_target in fidelity_targets:
            start = time.perf_counter()
            payload = truncate_statevector(tableau_state, fidelity_target)
            trunc_elapsed = time.perf_counter() - start
            rows.append({
                **metadata,
                'conversion': 'sv_truncate',
                'elapsed_s': trunc_elapsed,
                'success': True,
                'fidelity_target': float(fidelity_target),
                'achieved_fidelity': float(payload['fidelity']),
                'nonzero_amplitudes': int(payload['nonzero']),
            })
    else:
        for fidelity_target in fidelity_targets:
            rows.append({
                **metadata,
                'conversion': 'sv_truncate',
                'elapsed_s': None,
                'success': False,
                'fidelity_target': float(fidelity_target),
                'achieved_fidelity': None,
                'nonzero_amplitudes': None,
            })

    return rows


def run_conversion_benchmarks(
    qubits: Iterable[int],
    depths: Iterable[int],
    repeats: int,
    fidelity_targets: Sequence[float],
    *,
    seed: int = 1,
) -> List[Dict[str, object]]:
    rng = np.random.default_rng(seed)
    records: List[Dict[str, object]] = []
    for n in qubits:
        for depth in depths:
            for repeat in range(int(repeats)):
                records.extend(
                    _benchmark_single_configuration(
                        int(n), int(depth), repeat, fidelity_targets, rng=rng
                    )
                )
    return records


In [ ]:
qubit_sweep = [4, 6, 8, 10]
depth_sweep = [4, 8, 12]
repeats = 3
fidelity_targets = [0.99, 0.999]
seed = 7

records = run_conversion_benchmarks(qubit_sweep, depth_sweep, repeats, fidelity_targets, seed=seed)
len(records)


In [ ]:
if pd is not None:
    conversion_df = pd.DataFrame(records)
    display(conversion_df.head())
else:
    conversion_df = records
    conversion_df[:3]


In [ ]:
output_dir = Path('final_results')
output_dir.mkdir(parents=True, exist_ok=True)
output_csv = output_dir / 'conversion_microbenchmarks.csv'

if pd is not None:
    conversion_df.to_csv(output_csv, index=False)
else:
    import csv
    with output_csv.open('w', newline='', encoding='utf-8') as handle:
        writer = csv.DictWriter(handle, fieldnames=sorted(records[0].keys()))
        writer.writeheader()
        for row in records:
            writer.writerow(row)
print(f'Saved {output_csv}')


In [ ]:
if pd is not None:
    agg = (
        conversion_df[conversion_df['conversion'].isin(['tableau_to_sv', 'tableau_to_dd', 'dd_to_sv'])]
        .dropna(subset=['elapsed_s'])
        .groupby(['conversion', 'num_qubits', 'depth'])['elapsed_s']
        .median()
        .reset_index()
    )
else:
    from collections import defaultdict
    buckets: Dict[tuple, List[float]] = defaultdict(list)
    for row in records:
        if row['conversion'] in {'tableau_to_sv', 'tableau_to_dd', 'dd_to_sv'} and row.get('elapsed_s'):
            key = (row['conversion'], row['num_qubits'], row['depth'])
            buckets[key].append(row['elapsed_s'])
    agg = [
        {'conversion': conv, 'num_qubits': n, 'depth': d, 'elapsed_s': float(np.median(vals))}
        for (conv, n, d), vals in buckets.items()
    ]

fig, ax = plt.subplots()
if pd is not None:
    for conv, subset in agg.groupby('conversion'):
        ax.plot(subset['num_qubits'], subset['elapsed_s'], marker='o', label=conv)
else:
    for conv in sorted({row['conversion'] for row in agg}):
        xs = [row['num_qubits'] for row in agg if row['conversion'] == conv]
        ys = [row['elapsed_s'] for row in agg if row['conversion'] == conv]
        ax.plot(xs, ys, marker='o', label=conv)
ax.set_title('Median conversion runtime vs qubit count')
ax.set_xlabel('Qubits')
ax.set_ylabel('Runtime (s)')
ax.legend()
fig.tight_layout()
plot_path = Path('plots') / 'conversion_runtime_vs_qubits.png'
plot_path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(plot_path, dpi=150)
plot_path


In [ ]:
if pd is not None:
    trunc = conversion_df[conversion_df['conversion'] == 'sv_truncate'].dropna(subset=['elapsed_s'])
    fig, ax = plt.subplots()
    for target, subset in trunc.groupby('fidelity_target'):
        ax.scatter(subset['nonzero_amplitudes'], subset['elapsed_s'], label=f'target={target}')
    ax.set_xscale('log')
    ax.set_xlabel('Retained amplitudes')
    ax.set_ylabel('Truncation runtime (s)')
    ax.set_title('Fidelity-aware truncation cost')
    ax.legend()
    fig.tight_layout()
    trunc_plot = Path('plots') / 'conversion_truncation_runtime.png'
    fig.savefig(trunc_plot, dpi=150)
    trunc_plot
else:
    'Install pandas to visualise truncation behaviour.'
